In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
              nn.Conv2d(1, 8, 3, padding=0),
              nn.BatchNorm2d(8),
              nn.ReLU(inplace=True),
              nn.Dropout(p=0.09))
        self.conv2 = nn.Sequential(
              nn.BatchNorm2d(8),
              nn.Conv2d(8, 16, 3, padding=0),
              nn.ReLU(inplace=True),
              nn.Dropout(p=0.09))
        self.conv3 = nn.Sequential(
              nn.BatchNorm2d(16),
              nn.Conv2d(16, 32, 3, padding=0),
              nn.ReLU(inplace=True))
        self.conv4 = nn.Sequential(
              nn.BatchNorm2d(32),
              nn.Conv2d(32, 8, 1, padding=0),
              nn.ReLU(inplace=True),
              nn.MaxPool2d(kernel_size=2, stride=2))
        self.conv5 = nn.Sequential(
              nn.BatchNorm2d(8),
              nn.Conv2d(8, 8, 3, padding=0),
              nn.ReLU(inplace=True),
              nn.Dropout(p=0.09))
        self.conv6 = nn.Sequential(
              nn.BatchNorm2d(8),
              nn.Conv2d(8, 16, 3, padding=0),
              nn.ReLU(inplace=True),
              nn.Dropout(p=0.09))
        self.conv7 = nn.Sequential(
              nn.BatchNorm2d(16),
              nn.Conv2d(16, 32, 3, padding=0),
              nn.ReLU(inplace=True))
        self.conv8 = nn.Sequential(
              nn.Conv2d(32, 10, 1, padding=0))
        self.gap = nn.AdaptiveAvgPool2d((1,1))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
              ReLU-3            [-1, 8, 26, 26]               0
           Dropout-4            [-1, 8, 26, 26]               0
       BatchNorm2d-5            [-1, 8, 26, 26]              16
            Conv2d-6           [-1, 16, 24, 24]           1,168
              ReLU-7           [-1, 16, 24, 24]               0
           Dropout-8           [-1, 16, 24, 24]               0
       BatchNorm2d-9           [-1, 16, 24, 24]              32
           Conv2d-10           [-1, 32, 22, 22]           4,640
             ReLU-11           [-1, 32, 22, 22]               0
      BatchNorm2d-12           [-1, 32, 22, 22]              64
           Conv2d-13            [-1, 8, 22, 22]             264
             ReLU-14            [-1, 8,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
def train_eval(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(train_loader.dataset)

    print('\n\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))

In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    #train_eval(model, device, train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1017, Accuracy: 9696/10000 (96.96%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0656, Accuracy: 9804/10000 (98.04%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0260, Accuracy: 9916/10000 (99.16%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0252, Accuracy: 9920/10000 (99.20%)



loss=0.019745895639061928 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.91it/s]



Test set: Average loss: 0.0204, Accuracy: 9938/10000 (99.38%)

